In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
#Todo esto es la inicializacion, siempre necesitaremos uns parkcontext desde el cual empezaremos a trabajar


In [2]:
#Reduce(operacion)
#Combina los elementos utilizando un operador

In [10]:
rdd = sc.parallelize([1,2,3],3)

In [11]:
rdd.collect()

[1, 2, 3]

In [12]:
rdd.reduce(lambda x,y: x+y) #CReamos con lambda una operacion para reducir los elementos, en este caso estariamos haciendo
#una suma de los elementos

6

In [13]:
rdd.reduce(lambda x,y: x+y**2) # esto haria (1 + 2**2) + 3**2 

14

In [14]:
#En vez de usar una funcion lambda podemos usar un operador por ejemplo 
rdd = sc.parallelize([1,2,3],3)
from operator import add
rdd.reduce(add)

6

In [15]:
#En python tenemos varios operadores como add para sumar add, restar sub, multiplicar mul...
from operator import mul
rdd.reduce(mul)

6

In [25]:
from operator import sub
rdd.reduce(sub)

3

In [4]:
from operator import sub,mul,add #Podemos importar todo de golpe


In [ ]:
#Hay mucho operadores que podemos usar, pow() para elevar un numero a otro, el modulo con MOD()... ETC

In [23]:
#No funciona con rdds vacios si intentamos hacer un reduce de un RDD vacio 
#dará error

In [23]:
sc.parallelize([]).reduce(add)

ValueError: Can not reduce() empty RDD

In [12]:

#FOLD(valorCero,operacion)
#fold lo que hace es similar, solo que le añadimos un valor inicial para la operacion en cada particion 
#y luego al final otra vez

rdd = sc.parallelize([1,2,3],3)


rdd.fold(0,add)

6

In [37]:
rdd.fold(0,add) #en casos como este, si le añadimos un 0 va a devolver el mismo resultado, porque sumar 0 no afecta

6

In [38]:
rdd.fold(0,mul) #En casos de multiplicacion si le ponemos un 0, nos dara 0 ya que cualquier numero por 0 es 0

0

In [39]:
rdd.fold(1,mul) #Pero si le ponemos un 1, seria lo mismo que hacer una multiplcacion con reduce
#O sea para suma el 0 es neutro y para mul el 1 es neutro.. pero si lo cambiamos... ya si que nos afecta

6

In [ ]:
#pero si hacemos una suma y le ponemos un 1

In [5]:
rdd = sc.parallelize([1,2,3],3)
rdd.getNumPartitions()


3

In [6]:
rdd.fold(1,add)

10

In [13]:
#¿pq DA 10? haria 1(el valor qu le damos) + 1 = 2 -> 2+1 =3 -> 3+2. -> 5 +1 -> 6. 6+3 = 9 -> 9+1 = 10
rdd.getNumPartitions()

3

In [ ]:
#un caso de uso interesante que vimos en clase es el siguiente
#Un RDD de pujas, en el que nos interesa marcar el valor minimo para la puja, por ejemplo
#Estamos vendiendo un producto y queremos venderlo, pero no por menos de 300, entonces nos interesa sacar el valor máximo pero 
#a su vez supere 300


In [55]:
#vale por ejemoplo:
RDD_puja = sc.parallelize([400,345,122,30,56,90])

In [56]:
RDD_puja.fold(300,lambda x,y: max(x,y)) # Saca el valor maximo mayor de 300

400

In [57]:
RDD_puja.fold(500,lambda x,y: max(x,y)) # Saca el valor maximo mayor de 500, si no hay saca 500

500

In [ ]:
#Como se calcula tambien sobre el valor inicial, en este caso lo podemos utilizar como punto de partida para nuestra puja.

#Por lo general utilizaremos reduce, ya que nos devulve el resultado normal por asi decirlo, pero en caso concretos como el
#ejemplo que hemos visto de la puja, es interesante utilizarlo, pero no se suele usar


# AGEGATE(valorCero,seqOp,CombOp)
#Tiene 3 parametros


#ZeroValue - El valor inicial

#secOp - es para acumular los resultados por particion

#CombOP - es para acumular los resultados de todas las particiones


In [16]:
rdd = sc.parallelize(range(1,6),3) #el tres es el numero de particiones
#Si queremos ver el numero de particiones que tenemos:
rdd.getNumPartitions()

3

In [17]:
#si queremos ver como se distribuyen las particiones:
rdd.glom().collect()

[[1], [2, 3], [4, 5]]

In [27]:
#(1) (2 3) (4 5) #lo que tenemos
#Vamos a hacer 
rdd.aggregate(0,lambda x,y:add(x,y),lambda x,y:sub(x,y))

-15

In [74]:
#la primera operacion nos dice que para cada particion se sume:
#    (1) (2 + 3) (4 + 5) -> 1 5 9
#El segundo argumento nos dice que restemos:
#    -(1)  -(2 + 3) - (4 + 5) 

In [25]:
-(1)  -(5) - (9)

-15

In [77]:
#este agregate se usa poco, pero bueno para que sepais que existe y que se puede usar, para
#hacer logicas algo complejas

In [49]:
#Si tenemos nuestro RDD particionado por ejemplo por dias
#tenemos una lista de compra por dias, como en los ejemplos anteriores que hemos visto, y 
# necesitamos al total de la compra de cada dia sumarle un impuesto del 10%, seria interesante esta agregacion no?

gastos_1 = [5,5,10,5,20,5]
gastos_2 = [5,10,50,35]

rdd_gastos = sc.parallelize(gastos_1,1).union(sc.parallelize(gastos_2,1))

In [50]:
rdd_gastos.collect()

[5, 5, 10, 5, 20, 5, 5, 10, 50, 35]

In [51]:
rdd_gastos.glom().collect()

[[5, 5, 10, 5, 20, 5], [5, 10, 50, 35]]

In [54]:
agregado_con_impuesto = rdd_gastos.aggregate(0,lambda x,y:add(x,y),lambda x,y:add(x,(y+(y*0.1))))

#El resultado tiene que ser...
#primero suma lo que hay en el primer grupo y da 50 luego suma lo del segundo grupo y da 100
#en la ultima parte sera sumar grupos, primero sumará Nada con el primer grupo + el porcentaje.. o sea 0+(50+5)
#y luego sumara eso mas el segundo grupo mas el porcentaje 55 + (100+10) -> resultado 165

agregado_con_impuesto

165.0